<a href="https://colab.research.google.com/github/Bobbyorr007/GRADIENT-TAPE-BASICS/blob/main/GRADIENT_TAPE_BASICS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**IMPORTS**

In [3]:
import tensorflow as tf

**EXERCISE ON THE BASICS OF GRADIENT TAPE**


Using tf.GradientTape() for automatic differentiations.

In [4]:
# Define a 2x2 array of ones
x = tf.ones((2,2))

with tf.GradientTape() as t:
  #Record the action performed on tensor x with watch
  t.watch(x)

  #Define y as the sum of elements in x
  y = tf.reduce_sum(x)

  #Let z be the square of y
  z = tf.square(y)

#Now we get the differentiation of z with respect to the original tensor x
dz_dx = t.gradient(z,x)

# Print out the result
dz_dx


<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[8., 8.],
       [8., 8.]], dtype=float32)>

**BY DEFAULT GRADIENT TAPE EXPIRES AFTER ONE USE**

Gradient Tape is set to (persistence=False) by default, this makes it expires after one usage of calculating differentiation.
if you want to use Gradient Tape for multiple gradient calculation, it should be set to (persistence=True).

In [5]:
x = tf.ones((2,2))

# Set (persistence = True) so that the Tape can be reused.
with tf.GradientTape(persistent=True) as t:
  t.watch(x)

  y = tf.reduce_sum(x)

  z = tf.square(y)

# Compute dz_dx.
dz_dx = t.gradient(z,x)
dz_dx

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[8., 8.],
       [8., 8.]], dtype=float32)>

**Now that  I set persistence=True, we can reuse the Tape**


Lets try computing another gradient

In [6]:
dy_dx = t.gradient(y,x)
dy_dx

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[1., 1.],
       [1., 1.]], dtype=float32)>

To stop the Tape when you don't need it again, i.e when you are done computing gradient we use this command;

In [7]:
# Drop the reference to t
del t

**NESTED GRADIENT TAPES**

I computted a higher order derivaties by nesting the Gradient Tape:

***These are the acceptable indentation of the first gradient calculation***


The first gradient calculation should at least be inside the outer with block.

In [9]:
x = tf.Variable(3.0)
with tf.GradientTape() as tape_2:
  with tf.GradientTape() as tape_1:
    y = x * x * x

  # The first gradient calculation should at least occur within the outer with block.
  dy_dx = tape_1.gradient(y,x)
d2y_d2x = tape_2.gradient(dy_dx, x)

print(dy_dx)
print(d2y_d2x)

tf.Tensor(27.0, shape=(), dtype=float32)
tf.Tensor(18.0, shape=(), dtype=float32)


**Where not to indent the first gradient calculation**

If the first gradient calculation is indented outside the outer with block, it would work but wont persist for the second gradient calculation. Even if persistence is set to true, this wouldnt work.


**Where to indent the second gradient calculation**

The second gradient calcultion should be indented at the same indentation or  tab to the left from the first gradient calculation or more.

It shouldnt be indented more than the first gradient calculation.

This is acceptable;

  dy_dx = tape_1.gradient(y,x)

  d2y_d2x = tape_2.gradient(dy_dx, x)

 **OR**

      dy_dx = tape_1.gradient(y,x)

d2y_d2x = tape_2.gradient(dy_dx, x)

**NOT**

  dy_dx = tape_1.gradient(y,x)

      d2y_d2x = tape_2.gradient(dy_dx, x)